In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from pathlib import Path
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
#from scipy.signal import sosfiltfilt, butter
from dtw import dtw
#from dtaidistance import dtw_visualisation as dtwvis


import sys
sys.path.append("../src")
from fast_open_data import open_data_filter
from data_conversions import transition_matrix, sep_modo

In [10]:
def estimates(list_voluntary):
    f_name = 'C:/Users/User/OneDrive/TCC/ema_motion_analysis_imu/data/data_angle/Transitions/'
    sensor = ['_S_truth', '_R_truth', '_L_truth']
    #list_voluntary = [5, 6, 8, 13, 15, 16, 21, 23, 24, 25, 27, 28, 29, 30, 31, 34, 36, 38, 39, 40, 41, 
    #                   45, 47, 49, 51, 52, 55, 57, 60, 65, 66, 67, 68, 69, 71, 75, 78, 80, 81, 86, 87]
    est_manual = []

    for i in list_voluntary:
        for j in range(len(sensor)):
            data_path = f_name + str(i) + sensor[j]
            est_manual.append(np.loadtxt(data_path, delimiter=';'))
    return est_manual

In [11]:
#f_name = 'C:/Users/User/Desktop/Testes/'
#list_voluntary = [5, 6, 8]
#sensor = ['_S_truth', '_R_truth', '_L_truth']
#est_manual = []

#for i in list_voluntary:
#    for j in range(len(sensor)):
#        data_path = f_name + str(i) + sensor[j]
#        est_manual.append(np.loadtxt(data_path, delimiter=';'))

In [164]:
def extract_metrics(est_manual):

    t = []
    dif_temp_sit_SR = []
    dif_temp_sit_SL = []
    dif_temp_rise_SR = []
    dif_temp_rise_SL = []
    seg_sit_SR = []
    seg_sit_SL = []
    seg_rise_SR = []
    seg_rise_SL = []
     
    S_ref, R_ref, L_ref = open_data_filter(52, key = 'S1_Synched')
    freq_amostragem = 120
    for i in range(len(list_voluntary)):
        
        S_alvo, R_alvo, L_alvo = open_data_filter(list_voluntary[i], key = 'S1_Synched')
                
        # Criando vetor de tempo
        t_alvo = np.linspace(0, len(S_alvo) / freq_amostragem, len(S_alvo))
        
        # Aplicando as transformações do DTW para pescoço, perna direita e perna esquerda
        distance_R, paths_R = fastdtw(R_ref, R_alvo, radius=1, dist=euclidean)
        distance_L, paths_L = fastdtw(L_ref, L_alvo, radius=1, dist=euclidean)
        distance_S, paths_S = fastdtw(S_ref, S_alvo, radius=1, dist=euclidean)

        paths_S = np.array(paths_S)
        paths_R = np.array(paths_R)
        paths_L = np.array(paths_L)
        
        
        est_DTW_S, est_DTW_R, est_DTW_L = relationship_ref_alvo(paths_S, paths_R, paths_L)
        
        
        est_manual_S = est_manual[i*3]
        est_manual_R = est_manual[i*3 + 1]
        est_manual_L = est_manual[i*3 + 2]
                
        est_sit_manual_SR, est_sit_manual_SL = est_sit(est_manual_S, est_manual_R, est_manual_L)
        est_sit_DTW_SR, est_sit_DTW_SL = est_sit(est_DTW_S, est_DTW_R, est_DTW_L)
        est_rise_manual_SR, est_rise_manual_SL = est_rise(est_manual_S, est_manual_R, est_manual_L)
        est_rise_DTW_SR, est_rise_DTW_SL = est_rise(est_DTW_S, est_DTW_R, est_DTW_L)
             
        
        dif_sit_SR = calc_dif(est_sit_manual_SR, est_sit_DTW_SR, t_alvo)
        dif_sit_SL = calc_dif(est_sit_manual_SL, est_sit_DTW_SL, t_alvo)
        dif_rise_SR = calc_dif(est_rise_manual_SR, est_rise_DTW_SR, t_alvo)
        dif_rise_SL = calc_dif(est_rise_manual_SL, est_rise_DTW_SL, t_alvo)
        
        seg_time_sit_SR = calc_seg_time(est_sit_manual_SR, est_sit_DTW_SR, t_alvo)
        seg_time_sit_SL = calc_seg_time(est_sit_manual_SL, est_sit_DTW_SL, t_alvo)
        seg_time_rise_SR = calc_seg_time(est_rise_manual_SR, est_rise_DTW_SR, t_alvo)
        seg_time_rise_SL = calc_seg_time(est_rise_manual_SL, est_rise_DTW_SL, t_alvo)
        
        
        dif_temp_sit_SR.append(dif_sit_SR) 
        dif_temp_sit_SL.append(dif_sit_SL) 
        dif_temp_rise_SR.append(dif_rise_SR) 
        dif_temp_rise_SL.append(dif_rise_SL)
        
        seg_sit_SR.append(seg_time_sit_SR)
        seg_sit_SL.append(seg_time_sit_SL)
        seg_rise_SR.append(seg_time_rise_SR)
        seg_rise_SL.append(seg_time_rise_SL)  
                
        print('Voluntário ' + str(list_voluntary[i]) + ' OK!')
        
    dif_temp_sit_SR = np.array(dif_temp_sit_SR)
    dif_temp_sit_SL = np.array(dif_temp_sit_SL)
    dif_temp_rise_SR = np.array(dif_temp_rise_SR)
    dif_temp_rise_SL = np.array(dif_temp_rise_SL)
    
    seg_sit_SR = np.array(seg_sit_SR)
    seg_sit_SL = np.array(seg_sit_SL)
    seg_rise_SR = np.array(seg_rise_SR)
    seg_rise_SL = np.array(seg_rise_SL)

    return dif_temp_sit_SR, dif_temp_sit_SL, dif_temp_rise_SR, dif_temp_rise_SL, seg_sit_SR, seg_sit_SL, seg_rise_SR, seg_rise_SL

In [176]:
def calc_seg_time(est_manual, est_DTW, t):
    
    seg_time_fase2_manual = t[est_manual[1]] - t[est_manual[0]]
    seg_time_fase3_manual = t[est_manual[2]] - t[est_manual[1]]
    
    seg_time_fase2_DTW = t[est_DTW[1]] - t[est_DTW[0]]
    seg_time_fase3_DTW = t[est_DTW[2]] - t[est_DTW[1]]
    
    seg_time = np.c_[seg_time_fase2_manual, seg_time_fase3_manual, seg_time_fase2_DTW, seg_time_fase3_DTW]
    
    return seg_time

In [177]:
def calc_dif(est_manual, est_DTW, t):
    dif_temp = t[est_manual] - t[est_DTW]
    return dif_temp

In [178]:
def est_rise(est_S, est_R, est_L):
    
    sep = int((est_S[2] + est_S[3])/2)
    
    est_rise_SR = np.array([est_S[3]-sep, est_S[4]-sep, min(est_S[5], est_R[3])-sep])
    est_rise_SR = np.sort(est_rise_SR)
    est_rise_SL = np.array([est_S[3]-sep, est_S[4]-sep, min(est_S[5], est_L[3])-sep])
    est_rise_SL = np.sort(est_rise_SL)
    
    est_rise_SR = est_rise_SR.astype('int')
    est_rise_SL = est_rise_SL.astype('int')
    
    return  est_rise_SR, est_rise_SL   

In [179]:
def est_sit(est_S, est_R, est_L):
    est_sit_SR = np.array([max(est_S[0], est_R[0]), est_S[1], est_S[2]])
    est_sit_SR = np.sort(est_sit_SR)
    est_sit_SL = np.array([max(est_S[0], est_L[0]), est_S[1], est_S[2]])
    est_sit_SL = np.sort(est_sit_SL)
    
    est_sit_SR = est_sit_SR.astype('int')
    est_sit_SL = est_sit_SL.astype('int')
    
    return  est_sit_SR, est_sit_SL   

In [180]:
def relationship_ref_alvo(paths_S, paths_R, paths_L):
    
    est_ref_R = [ 100, 320, 605, 790]
    est_ref_L = [ 30, 320, 605, 740]
    est_ref_S = [ 145, 263, 365, 570, 647, 751]
    
    est_S = []
    for i in est_ref_S:
        idx = np.where(paths_S[:,0]==i)    
        est_S.append(int(np.mean(paths_S[idx,1])))
    if est_S[2] == est_S[3]:
        est_S[2] = est_S[2]-3
        est_S[3] = est_S[3]+3

    est_L = []
    for i in est_ref_L:
        idx = np.where(paths_L[:,0]==i)    
        est_L.append(int(np.mean(paths_L[idx,1])))

    est_R = []
    for i in est_ref_R:
        idx = np.where(paths_R[:,0]==i)    
        est_R.append(int(np.mean(paths_R[idx,1])))
        
    
    return est_S, est_R, est_L

In [192]:
list_voluntary = [5, 6, 8, 13, 15, 16, 21, 23, 24, 25, 27, 28, 29, 30, 31, 34, 36, 38, 39, 40, 41, 
                  45, 47, 49, 51, 52, 55, 57, 60, 65, 66, 67, 68, 69, 71, 75, 78, 80, 81, 86, 87]

In [253]:
list_voluntary = [23, 52]
est_manual = estimates(list_voluntary)
t_sit_SR, t_sit_SL, t_rise_SR, t_rise_SL, f_sit_SR, f_sit_SL, f_rise_SR, f_rise_SL = extract_metrics(est_manual)

Voluntário 23 OK!
Voluntário 52 OK!


In [257]:
print('-------------------------------------------------------------------------------------------------------------------')
print('                                        SENTAR (TRONCO E PERNA DIREITA)                                            ')
print('')
tempo_sit_fase_2_manual_SR = np.mean(f_sit_SR[:,:,0])
print('Tempo médio para fase II - Segmentação Manual', 
      tempo_sit_fase_2_manual_SR, 'segundos')
tempo_sit_fase_2_DTW_SR = np.mean(f_sit_SR[:,:,2])
print('Tempo médio para fase II - Segmentação -> DTW', 
      tempo_sit_fase_2_DTW_SR, 'segundos')

tempo_sit_fase_3_manual_SR = np.mean(f_sit_SR[:,:,1])
print('Tempo médio para fase III - Segmentação Manual', 
      tempo_sit_fase_3_manual_SR, 'segundos')
tempo_sit_fase_3_DTW_SR = np.mean(f_sit_SR[:,:,3])
print('Tempo médio para fase III - Segmentação -> DTW', 
      tempo_sit_fase_3_DTW_SR, 'segundos')
print('-------------------------------------------------------------------------------------------------------------------')
print('                                        SENTAR (TRONCO E PERNA ESQUERDA)                                           ')
print('')
tempo_sit_fase_2_manual_SL = np.mean(f_sit_SL[:,:,0])
print('Tempo médio para fase II - Segmentação Manual', 
      tempo_sit_fase_2_manual_SL, 'segundos')
tempo_sit_fase_2_DTW_SL = np.mean(f_sit_SL[:,:,2])
print('Tempo médio para fase II - Segmentação -> DTW', 
      tempo_sit_fase_2_DTW_SL, 'segundos')

tempo_sit_fase_3_manual_SL = np.mean(f_sit_SL[:,:,1])
print('Tempo médio para fase III - Segmentação Manual', 
      tempo_sit_fase_3_manual_SL, 'segundos')
tempo_sit_fase_3_DTW_SL = np.mean(f_sit_SL[:,:,3])
print('Tempo médio para fase III - Segmentação -> DTW', 
      tempo_sit_fase_3_DTW_SL, 'segundos')
print('-------------------------------------------------------------------------------------------------------------------')
print('                                        LEVANTAR (TRONCO E PERNA DIREITA)                                          ')
print('')
tempo_rise_fase_2_manual_SR = np.mean(f_rise_SR[:,:,0])
print('Tempo médio para fase II - Segmentação Manual', 
      tempo_rise_fase_2_manual_SR, 'segundos')
tempo_rise_fase_2_DTW_SR = np.mean(f_rise_SR[:,:,2])
print('Tempo médio para fase II - Segmentação -> DTW', 
      tempo_rise_fase_2_DTW_SR, 'segundos')

tempo_rise_fase_3_manual_SR = np.mean(f_rise_SR[:,:,1])
print('Tempo médio para fase III - Segmentação Manual', 
      tempo_rise_fase_3_manual_SR, 'segundos')
tempo_rise_fase_3_DTW_SR = np.mean(f_rise_SR[:,:,3])
print('Tempo médio para fase III - Segmentação -> DTW', 
      tempo_rise_fase_3_DTW_SR, 'segundos')
print('-------------------------------------------------------------------------------------------------------------------')
print('                                        LEVANTAR (TRONCO E PERNA ESQUERDA)                                         ')
print('')
tempo_rise_fase_2_manual_SL = np.mean(f_rise_SL[:,:,0])
print('Tempo médio para fase II - Segmentação Manual', 
      tempo_rise_fase_2_manual_SL, 'segundos')
tempo_rise_fase_2_DTW_SL = np.mean(f_rise_SL[:,:,2])
print('Tempo médio para fase II - Segmentação -> DTW', 
      tempo_rise_fase_2_DTW_SL, 'segundos')

tempo_rise_fase_3_manual_SL = np.mean(f_rise_SL[:,:,1])
print('Tempo médio para fase III - Segmentação Manual', 
      tempo_rise_fase_3_manual_SL, 'segundos')
tempo_rise_fase_3_DTW_SL = np.mean(f_rise_SL[:,:,3])
print('Tempo médio para fase III - Segmentação -> DTW', 
      tempo_rise_fase_3_DTW_SL, 'segundos')

-------------------------------------------------------------------------------------------------------------------
                                        SENTAR (TRONCO E PERNA DIREITA)                                            

Tempo médio para fase II - Segmentação Manual 0.9218622647359925 segundos
Tempo médio para fase II - Segmentação -> DTW 0.7883667977517032 segundos
Tempo médio para fase III - Segmentação Manual 0.8718021567507972 segundos
Tempo médio para fase III - Segmentação -> DTW 1.0511727517029568 segundos
-------------------------------------------------------------------------------------------------------------------
                                        SENTAR (TRONCO E PERNA ESQUERDA)                                           

Tempo médio para fase II - Segmentação Manual 1.0011098751821081 segundos
Tempo médio para fase II - Segmentação -> DTW 1.2513971876599599 segundos
Tempo médio para fase III - Segmentação Manual 0.8718021567507972 segundos
Tempo médio p

In [255]:
print('-------------------------------------------------------------------------------------------------------------------')
print('                                        SENTAR (TRONCO E PERNA DIREITA)                                            ')
print('')
fase_1_2_sit_manual_SR = np.mean(abs(t_sit_SR[:,0]))
print('Média do erro na detecção da transição FaseI/FaseII', round(fase_1_2_sit_manual_SR, 4))
fase_2_3_sit_manual_SR = np.mean(abs(t_sit_SR[:,1]))
print('Média do erro na detecção da transição FaseII/FaseIII', round(fase_2_3_sit_manual_SR, 4))
fase_3_4_sit_manual_SR = np.mean(abs(t_sit_SR[:,2]))
print('Média do erro na detecção da transição FaseIII/FaseIV', round(fase_3_4_sit_manual_SR, 4))
print('-------------------------------------------------------------------------------------------------------------------')
print('                                        SENTAR (TRONCO E PERNA ESQUERDA)                                           ')
print('')
fase_1_2_sit_manual_SL = np.mean(abs(t_sit_SL[:,0]))
print('Média do erro na detecção da transição FaseI/FaseII', round(fase_1_2_sit_manual_SL, 4))
fase_2_3_sit_manual_SL = np.mean(abs(t_sit_SL[:,1]))
print('Média do erro na detecção da transição FaseII/FaseIII', round(fase_2_3_sit_manual_SL, 4))
fase_3_4_sit_manual_SL = np.mean(abs(t_sit_SL[:,2]))
print('Média do erro na detecção da transição FaseIII/FaseIV', round(fase_3_4_sit_manual_SL, 4))
print('-------------------------------------------------------------------------------------------------------------------')
print('                                        LEVANTAR (TRONCO E PERNA DIREITA)                                          ')
print('')
fase_1_2_rise_manual_SR = np.mean(abs(t_rise_SR[:,0]))
print('Média do erro na detecção da transição FaseI/FaseII', round(fase_1_2_rise_manual_SR, 4))
fase_2_3_rise_manual_SR = np.mean(abs(t_rise_SR[:,1]))
print('Média do erro na detecção da transição FaseII/FaseIII', round(fase_2_3_rise_manual_SR, 4))
fase_3_4_rise_manual_SR = np.mean(abs(t_rise_SR[:,2]))
print('Média do erro na detecção da transição FaseIII/FaseIV', round(fase_3_4_rise_manual_SR, 4))
print('-------------------------------------------------------------------------------------------------------------------')
print('                                        LEVANTAR (TRONCO E PERNA ESQUERDA)                                         ')
print('')
fase_1_2_rise_manual_SL = np.mean(abs(t_rise_SL[:,0]))
print('Média do erro na detecção da transição FaseI/FaseII', round(fase_1_2_rise_manual_SL, 4))
fase_2_3_rise_manual_SL = np.mean(abs(t_rise_SL[:,1]))
print('Média do erro na detecção da transição FaseII/FaseIII', round(fase_2_3_rise_manual_SL, 4))
fase_3_4_rise_manual_SL = np.mean(abs(t_rise_SL[:,2]))
print('Média do erro na detecção da transição FaseIII/FaseIV', round(fase_3_4_rise_manual_SL, 4))

-------------------------------------------------------------------------------------------------------------------
                                        SENTAR (TRONCO E PERNA DIREITA)                                            

Média do erro na detecção da transição FaseI/FaseII 0.3838
Média do erro na detecção da transição FaseII/FaseIII 0.0083
Média do erro na detecção da transição FaseIII/FaseIV 0.171
-------------------------------------------------------------------------------------------------------------------
                                        SENTAR (TRONCO E PERNA ESQUERDA)                                           

Média do erro na detecção da transição FaseI/FaseII 0.267
Média do erro na detecção da transição FaseII/FaseIII 0.0083
Média do erro na detecção da transição FaseIII/FaseIV 0.171
-------------------------------------------------------------------------------------------------------------------
                                        LEVANTAR (TRONCO E 

In [246]:
round(fase_3_4_rise_manual_SL, 4)

0.2911